In [9]:
import pandas as pd
import geopandas as gpd
import sqlalchemy as sa
import pyodbc
import tkinter as tk
from sqlalchemy import create_engine
from api_config import geocode_postcode, fetch_isochrone, PostcodeNotFound
from geo_config import isochrone_to_gdf, teams_to_gdf, filter_teams_by_minutes

In [10]:
# Define custom colour palette

humannature = {'GC Dark Green': '#294238',
               'GC Light Green': '#b2d235',
               'GC Mid Green': '#50b748',
               'GC Orange': '#f57821',
               'GC Light Grey': '#e6ebe3'
               }

In [11]:
# SQL server details 
server = 'dblistener1'
database = 'information_centre'
driver = 'ODBC Driver 17 for SQL Server' # Create SQLAlchemy engine and install necessary drivers
connection_string = f'mssql+pyodbc://{server}/{database}?driver={driver}'
engine = create_engine(connection_string)

# SQL query to select dataset from SSMS tables and save into allvisits dataframe
query = """
SELECT
    a.intContractorID,
    a.strName AS Contractor,
    e.BusinessUnitID,
    e.Name AS BusinessUnit,
    UPPER(a.strPostcode) AS Postcode,
    g.Latitude,
    g.Longitude,
    a.InternalContractor
FROM
    tblContractor a
    LEFT JOIN Contractor_Business_Unit d
        ON a.intContractorID = d.ContractorID
    LEFT JOIN Business_Unit e
        ON d.BusinessUnitID = e.BusinessUnitID
    LEFT JOIN Business_Unit_Master_Status f
        ON d.StatusID = f.StatusID
    LEFT JOIN dbs_PostCode.dbo.tblPostcodes_New g
        ON REPLACE(a.strPostcode, ' ', '') = g.PostcodeNoSpaces COLLATE Latin1_General_CI_AS
WHERE
    ISNULL(a.bDisabled, 0) = 0
    AND f.StatusID IN (60, 70, 80)
    AND ISNULL(a.IsTest, 0) = 0
    AND e.BusinessUnitID != 37;
"""
with engine.connect() as connection:
    fieldteams = pd.read_sql(query, connection)
    connection.close()

In [ ]:
try:
    site_lon, site_lat = geocode_postcode("CM12 0EQ")  # or the user-entered postcode
except PostcodeNotFound as e:
    # In the GUI: show a messagebox with str(e) and return early
    print(f"Postcode error: {e}")
else:
    # One API call – returns 15/30/45/60 minute contours
    iso_geojson = fetch_isochrone(site_lon, site_lat)

    # Convert once; filter locally
    iso_gdf   = isochrone_to_gdf(iso_geojson)
    teams_gdf = teams_to_gdf(fieldteams)

    # 60 minutes = one hour filter
    filtered_teams = filter_teams_by_minutes(teams_gdf, iso_gdf, minutes=60)


,intContractorID,Contractor,BusinessUnitID,BusinessUnit,Postcode,Latitude,Longitude,InternalContractor,geometry
0,8028,Oakwood and GM Tree Services Ltd,7,Grounds Maintenance,CO59PY,51.843273,0.714097,0,POINT (0.7141 51.84327)
1,8028,Oakwood and GM Tree Services Ltd,33,GM & Pest Control,CO59PY,51.843273,0.714097,0,POINT (0.7141 51.84327)
2,8028,Oakwood and GM Tree Services Ltd,1,Arboriculture Amenity,CO59PY,51.843273,0.714097,0,POINT (0.7141 51.84327)
3,3244,Planteria,13,Specialist Contractor,CM226AA,51.924541,0.245823,0,POINT (0.24582 51.92454)
4,7995,Green Net Eco Ltd,3,Arboriculture Utility,CM1 3ST,51.739994,0.415405,0,POINT (0.4154 51.73999)


Teams within 60 minutes: 135
